In [ ]:
!pip install transformers evaluate rouge_score pyemd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

%cd drive/MyDrive/CS\ 224N/CS\ 224N\ Project
%ls # verify that you are in the right directory

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1x7bmzM_qtbj3CPKzenuwvwocK9KiZzWU/CS 224N Project
'224N Experiments (GAN on 998 Samples).gsheet'
'224N Project Brainstorm.gdoc'
'224N Project Helpful Tutorials.gdoc'@
'224N Project Milestone Notes.gdoc'
 Adversarial-T5_Structure_1.ipynb
 aita_clean.csv
 aita_comments.csv
 aita_test_set.csv
 aita_test_set.gsheet
 aita_train_set.csv
 aita_valid_set.csv
 banana/
 banana.ipynb
 bart-base-checkpoint-204000/
 bart-baseline-attempt2.ipynb
 bart-checkpoint-5000/
 bert-baseline.ipynb
 checkpoint.txt
 config.json
 csvs/
 dataset_agg.ipynb
 drive/
 Evaluate.ipynb
'experimenting with gumbel.ipynb'
 finetune-gpt2/
'first proposal OLD'/
 gan-gen-trial
 gpt-2-attempt2.ipynb
 gpt2-attempt3.ipynb
 gpt2-baseline.ipynb
 gpt2-small-rationale-generation/
 gpt2-wt-5.ipynb
 grape/
 grape-exploration.ipynb
 grape.ipynb
 logs/
 mango/
 Mango.ipynb
 mango-old/
 Orange.ipynb
 Pear.ipynb
 pineapple/
 Pineapple.ipynb
 pineapple-old/
 processed-set-gp

In [ ]:
import transformers
import evaluate
import pandas as pd
from pyemd import emd
# from gensim.models import KeyedVectors
# from gensim.test.utils import datapath
import gensim.downloader as api

from nltk.corpus import stopwords
from nltk import download

from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
)

# wmd pre processing steps
download('stopwords')
stop_words = stopwords.words('english')
wmd_model = api.load("glove-wiki-gigaword-100")

# EDIT THE MODELS YOU WANT TO TEST HERE
models = ['grape-final', 'pineapple-final', 'mango-final']


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[==================================================] 100.0% 128.1/128.1MB downloaded


In [ ]:
rouge = evaluate.load('rouge')
bleu = evaluate.load('bleu')

tokenizer = AutoTokenizer.from_pretrained(models[0])
test_df = pd.read_csv('aita_test_set.csv')[['text', 'comments']]
inputs = [test_df.iloc[i]['text'] for i in range(100)]
inputs = tokenizer(inputs, max_length=512, padding=True, truncation=True, return_tensors='pt')

In [ ]:
def wmd_formatter(input):
  new_list = []
  for string in input:
    words = string.split()
    filtered_words = [word for word in words if word not in stop_words]
    new_list.append(filtered_words)
  return new_list 

rouge_filename = 'rouge_scores.txt'
bleu_filename = 'bleu_scores.txt'
wmd_filename= 'wmd_scores.txt'

for model_name in models:
  model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
  preds = model.generate(**inputs, num_beams=8, do_sample=True, min_length=10, max_length=64)
  decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
  targets = [test_df.iloc[i]['comments'] for i in range(100)]
  rouge_results = rouge.compute(predictions=decoded_preds, references=targets)
  bleu_results = bleu.compute(predictions-decoded_preds, references=targets)
  with open(rouge_filename, 'a') as f:
    f.write(f'{model_name}: {rouge_results}\n')
  with open(bleu_filename, 'a') as f:
    f.write(f'{model_name}: {bleu_results}\n')
  
  # calculate wmd scores
  wmd_targets = wmd_formatter(targets)
  wmd_decoded_preds = wmd_formatter(decoded_preds)
  d_sum = 0
  for i in range(len(targets)):
    d = wmd_model.wmdistance(wmd_decoded_preds[i], wmd_targets[i])
    d_sum += d
  with open(wmd_filename, 'a') as w:
    w.write(f'{model_name}: {d_sum / len(targets)}\n')


In [ ]:


# targets = [test_df.iloc[i]['comments'] for i in range(2)]
# wmd_targets = wmd_formatter(targets)
# print(wmd_targets)
# wmd_decoded_preds = wmd_formatter(decoded_preds)
# print(wmd_decoded_preds)

# for i in range(len(targets)):
#   sentence_obama = 'Obama speaks to the media in Illinois'.lower().split()
#   sentence_president = 'The president greets the press in Chicago'.lower().split()
#   s = wmd_model.wmdistance(sentence_obama, sentence_president)
#   print(s)
#   print(wmd_decoded_preds[i], wmd_targets[i])
#   d = wmd_model.wmdistance(wmd_decoded_preds[i], wmd_targets[i])
#   print(d)
  

[['YTA.', "You're", 'wrong,', 'also', 'asked', 'opinion,', 'aggravated', 'already', 'difficult', 'situation', 'family.', 'You', 'help', 'anyone', 'comments', 'made', 'difference', "man's", 'behaviour.'], ['sleep', 'met']]
[['YTA.', 'Drunk', 'drivers', 'worst', 'people', 'world.'], ['NAH.', 'It’s', 'cross', 'necklace.', 'He', 'wears', 'sleep.', 'You’re', 'asshole', 'asking', 'wear', 'it.']]
3.4892687395218687
['YTA.', 'Drunk', 'drivers', 'worst', 'people', 'world.'] ['YTA.', "You're", 'wrong,', 'also', 'asked', 'opinion,', 'aggravated', 'already', 'difficult', 'situation', 'family.', 'You', 'help', 'anyone', 'comments', 'made', 'difference', "man's", 'behaviour.']
5.831665765644454
3.4892687395218687
['NAH.', 'It’s', 'cross', 'necklace.', 'He', 'wears', 'sleep.', 'You’re', 'asshole', 'asking', 'wear', 'it.'] ['sleep', 'met']
6.263320725377034


In [ ]:
# # calculate wmd scores
# for model_name in models:
#   wmd_scores = []
#   for i in range(100):
#     input_tokens = tokenizer.decode(inputs['input_ids'][i], skip_special_tokens=True)
#     print(input_tokens)
#     pred_tokens = decoded_preds[i].split()
#     target_tokens = targets[i].split()
#     wmd_input_pred = w2v_model.wmdistance(input_tokens, pred_tokens)
#     wmd_pred_target = w2v_model.wmdistance(pred_tokens, target_tokens)
#     wmd_input_target = w2v_model.wmdistance(input_tokens, target_tokens)
#     wmd_scores.append((wmd_input_pred, wmd_pred_target, wmd_input_target))
#   wmd_results = pd.DataFrame(wmd_scores, columns=['wmd_input_pred', 'wmd_pred_target', 'wmd_input_target'])
#   with open(wmd_filename, 'a') as f:
#     f.write(f'{model_name} WMD scores:\n')
#   wmd_results.to_csv(wmd_filename, mode='a', header=True)